# Yolo fine tuning notebook

## Summary 

Let's take one of the leading, heavily supported computer vision model families - YOLO - and apply it to the task of detecting and classifying animals in wildlife camera trap imagery.

We will be using YOLOv5 for this, prototyping and iterating with YOLOv5s and then finally using YOLOv5l or YOLOv5x for the final model. 

Based on experiments and prototypes locally, we will start with 640 res and 100 epochs but will progress as need be.

Depending on compute consumption we can also run a parameter sweep with wandb.

This notebook will focus on setting up our cloud computing instance so that we can allocate sufficient GPU or TPU resources to fine tuning our model.
Diagnostics and visualizations or training results and evaluation of performance will be done via wandb.

## Table of Contents

1. Setup our colab notebook environment
  - Mount google drive images
  - Select correct compute instance
  - Clone yolov5 repo
  - Prep python environment
  - Setup wandb logging
2. Train model
3. Assess performance using wandb diagnostics
4. Reiterate as needed


## Mount images folder from drive


In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
images_dir = '/content/drive/My Drive/wcsmall_bw'



## Or load files from local


In [3]:
import os
print(os.getcwd())
print(images_dir)

/content
/content/drive/My Drive/wcsmall_bw


## Setup our colab

### Set up colab environment

1. Switch to GPU runtime

2. Install and load dependencies

Hint: Make sure to stop the run time when it isn't being used!

In [4]:
!pip install wandb

     |████████████████████████████████| 1.8 MB 5.3 MB/s 
     |████████████████████████████████| 144 kB 48.5 MB/s 
     |████████████████████████████████| 181 kB 43.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=2c6d59c1d763f535f485750e00034a427945dd12904ca351db3a27e49755e367
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [6]:
# clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
import wandb
from IPython.display import Image, clear_output  # to display images


Cloning into 'yolov5'...
remote: Enumerating objects: 12783, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 12783 (delta 0), reused 0 (delta 0), pack-reused 12782
Receiving objects: 100% (12783/12783), 11.74 MiB | 24.28 MiB/s, done.
Resolving deltas: 100% (8915/8915), done.
/content/yolov5


In [7]:
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.0+cu111 (Tesla K80)


In [8]:
 wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Optional: Fix broken classes

You really shouldn't have to do this. But I have to, so I will show how.


In [9]:
print(os.getcwd())

/content/yolov5


In [12]:
#!python yolo_hardcoded_recoder.py -a /content/drive/MyDrive/datasets_lean/train/labels
#!python yolo_hardcoded_recoder.py -a /content/drive/MyDrive/datasets_lean/test/labels
#!python yolo_hardcoded_recoder.py -a /content/drive/MyDrive/datasets_lean/val/labels

9it [00:13,  1.95it/s]Class recoded!
14it [00:15,  2.48it/s]Class recoded!
15it [00:16,  2.56it/s]Class recoded!
20it [00:17,  3.47it/s]Class recoded!
24it [00:18,  3.68it/s]Class recoded!
34it [00:21,  3.59it/s]Class recoded!
43it [00:24,  2.84it/s]Class recoded!
62it [01:26,  3.36s/it]Class recoded!
Class recoded!
83it [01:26,  1.20it/s]Class recoded!
95it [01:26,  2.09it/s]Class recoded!
Class recoded!
Class recoded!
Class recoded!
115it [01:27,  4.58it/s]Class recoded!
127it [01:27,  6.98it/s]Class recoded!
137it [01:27,  9.64it/s]Class recoded!
Class recoded!
147it [01:27, 13.20it/s]Class recoded!
160it [01:27, 19.22it/s]Class recoded!
Class recoded!
Class recoded!
170it [01:27, 24.86it/s]Class recoded!
Class recoded!
180it [01:27, 31.10it/s]Class recoded!
Class recoded!
192it [01:27, 40.86it/s]Class recoded!
Class recoded!
Class recoded!
202it [01:28, 48.24it/s]Class recoded!
212it [01:28, 56.59it/s]Class recoded!
Class recoded!
223it [01:28, 65.80it/s]Class recoded!
Class recode

## Fine tuning

In [12]:
!python train.py --img 1280 --batch 4 --epochs 100 --data /content/yolov5/data/wcsmall_bw.yaml --weights yolov5s.pt 

wandb: Currently logged in as: hlydecker (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=, data=/content/yolov5/data/wcsmall_bw.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=4, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-124-g8c420c4 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1